# Training Process
This run has an example of running a training job

1. Creating AI template and instance

In [3]:
import os.path
import shutil

from superai.meta_ai.ai import AITemplate, AI, TrainingOrchestrator
from superai.meta_ai.parameters import Config, HyperParameterSpec, TrainingParameters, ModelParameters
from superai.meta_ai.schema import Schema

In [2]:
if os.path.exists(".AISave"):
    shutil.rmtree(".AISave")

In [19]:
template = AITemplate(
    input_schema=Schema(),
    output_schema=Schema(),
    configuration=Config(),
    name="MnistTrainingTemplate",
    description="Template of Sample MNIST training",
    model_class="MnistModel",
    requirements=["tensorflow-gpu==2.3.0", "polyaxon"],
    artifacts={"run": "resources/runDir/run_this.sh"},
    code_path=["resources/runDir"],
)
ai = AI(
    ai_template=template,
    input_params=template.input_schema.parameters(),
    output_params=template.output_schema.parameters(),
    name="mnist_training",
    version=1,
    description="AI instance of sample MNIST training",
)

[03/14/22 13:21:21] INFO     Copying all code_path content - MainThread             ]8;id=1647260481.18172-486147;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:273

                    INFO     Saved model in .AISave/mnist_training/1 - MainThread   ]8;id=1647260481.193953-363701;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:868

## Create and push training container
Use the following interface to create and push the training container.

> Note: Later, this interface will contain implementation to connect to meta-ai to create a training job.

In [4]:
ai._id = "DAAD6583-1FD7-4718-8B12-333701894FDB".lower()

hyperparams = HyperParameterSpec(epochs=10)
ai.training_deploy(
    orchestrator=TrainingOrchestrator.AWS_EKS,
    skip_build=False,
    enable_cuda=True,
    build_all_layers=True,
    training_parameters=TrainingParameters(hyperparameters=hyperparams)
)

[02/18/22 17:47:27] INFO     Base image                                            ]8;id=1645202847.4730282-361873;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:1208
                             'superai-model-s2i-python3711-gpu-seldon:1' found               
                             locally. - MainThread

                    INFO     No change in pip layer. Reusing old layers from image ]8;id=1645202847.482775-523639;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:1223
                             sha256:3702826942cecf12d4c049b8ad35753b59b8779e2fdfe6           
                             49687a521c64d44b31... - MainThread

                    INFO     Running 's2i build -E /Users/ankitmishra/Work/mysuper ]8;id=1645202847.4881501-538054;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:1303
                             ai/superai-sdk-private/docs/examples/ai/.AISave/mnist           
                             _training/1/environment -v                                      
                             /Users/ankitmishra/.aws:/root/.aws -v                           
                             /Users/ankitmishra/.superai:/root/.superai -v                   
                             /Users/ankitmishra/.canotic:/root/.canotic                      
                             --incremental=True .                                            
                             superai-model-s2i-python3711-gpu-seldon:1                       
                             mnist_training-pip-layer:1' - MainThread

[02/18/22 17:48:55] INFO     Running 's2i build -E /Users/ankitmishra/Work/mysuper ]8;id=1645202935.159571-544994;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:1303
                             ai/superai-sdk-private/docs/examples/ai/.AISave/mnist           
                             _training/1/environment -v                                      
                             /Users/ankitmishra/.aws:/root/.aws -v                           
                             /Users/ankitmishra/.superai:/root/.superai -v                   
                             /Users/ankitmishra/.canotic:/root/.canotic                      
                             --incremental=True . mnist_training-pip-layer:1                 
                             mnist_training:1' - MainThread

[02/18/22 17:48:56] INFO     Built main container `mnist_training:1` - MainThread  ]8;id=1645202936.395926-438812;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:1245

...

                    INFO     Logging in to ECR... - MainThread                ]8;id=1645202937.951545-289711;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/dockerizer/__init__.py\__init__.py]8;;\:246

[02/18/22 17:49:00] INFO     Tagging to `185169359328.dkr.ecr.us-east-1.amazo ]8;id=1645202940.725766-344046;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/dockerizer/__init__.py\__init__.py]8;;\:249
                             naws.com/models/dev/daad6583-1fd7-4718-8b12-3337                
                             01894fdb/mnist_training:1` - MainThread

                    INFO     Pushing image... - MainThread                    ]8;id=1645202940.7437272-273734;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/dockerizer/__init__.py\__init__.py]8;;\:252

Output()

[02/18/22 17:50:39] INFO      Image pushed successfully to 185169359328.dkr.e ]8;id=1645203039.829727-531090;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/dockerizer/__init__.py\__init__.py]8;;\:291
                             cr.us-east-1.amazonaws.com/models/dev/daad6583-1                
                             fd7-4718-8b12-333701894fdb/mnist_training:1  -                  
                             MainThread

# Starting training manually
To start training manually, in a separate terminal run, `polyaxon port-forward` to connect to Polyaxon API.

Then run the following cell
> Note: triggering training will be done from meta-ai in the future.

In [14]:
!cat polyaxonfile.yaml

version: 1.1
kind: component
tags: [examples, keras]

inputs:
- {name: conv1_size, type: int, value: 32, isOptional: true}
- {name: conv2_size, type: int, value: 64, isOptional: true}
- {name: dropout, type: float, value: 0.8, isOptional: true}
- {name: hidden1_size, type: int, value: 500, isOptional: true}
- {name: optimizer, type: str, value: adam, isOptional: true}
- {name: log_learning_rate, type: int, value: -3, isOptional: true}
- {name: epochs, type: int, value: 10, isOptional: true}

run:
  kind: job
  container:
    image: 185169359328.dkr.ecr.us-east-1.amazonaws.com/models/dev/daad6583-1fd7-4718-8b12-333701894fdb/mnist_training:1
#    workingDir: "{{ globals.artifacts_path }}/polyaxon-examples/in_cluster/keras/mnist"
    command: ["/opt/conda/envs/env/bin/superai", "ai", "method", "train"]
    imagePullPolicy: Always
    args: ["-p", "/home/model-server/",
           "-tp","/tmp",
           "-mp","/tmp",
           "-m","conv1_size={{ conv1_size }}",
           "-m","conv2_s

In [ ]:
!polyaxon run -f resources / polyaxonfile.yaml -u -l

Creating a new run...
A new run `3d56bfb861634b8e92cd11433e23b37a` was created
You can view this run on Polyaxon UI: http://localhost:8000/ui/default/dev/runs/3d56bfb861634b8e92cd11433e23b37a/
[================================] 958775/958775 - 00:00:00
Artifacts uploaded
Run is approved
Starting logs for run: <Name: None> - <uuid: 3d56bfb861634b8e92cd11433e23b37a>
warning
running
2022-02-18 17:51:29.912739+01:00 | Reading configs from /opt/conda/envs/env/lib/python3.7/site-packages/superai/settings.yaml
2022-02-18 17:51:29.912767+01:00 | Available envs:
2022-02-18 17:51:29.912770+01:00 | - local
2022-02-18 17:51:29.912773+01:00 | - dev
2022-02-18 17:51:29.912775+01:00 | - sandbox
2022-02-18 17:51:29.912777+01:00 | - stg
2022-02-18 17:51:29.912779+01:00 | - prod
2022-02-18 17:51:29.912781+01:00 | Reading configs from /opt/conda/envs/env/lib/python3.7/site-packages/superai/settings.yaml
2022-02-18 17:51:29.912783+01:00 | [02/18/22 16:51:29] INFO     Setting config : prod - MainThread    

2022-02-18 17:51:34.033765+01:00 | Epoch 1/10
2022-02-18 17:51:34.033863+01:00 | 2022-02-18 16:51:34.033782: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-02-18 17:51:34.033904+01:00 | 2022-02-18 16:51:34.033844: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1441] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
  1/600 [..............................] - ETA: 0s - loss: 2.3247 - accuracy: 0.0700[02/18/22 16:51:34] WARNING  From /opt/conda/envs/env/lib/pyt deprecation.py:323
2022-02-18 17:51:34.167184+01:00 |                              hon3.7/site-packages/tensorflow/
2022-02-18 17:51:34.167200+01:00 |                              python/ops/summary_ops_v2.py:127
2022-02-18 17:51:34.167204+01:00 |                              7: stop (from tensorflow.python.
2022-02-18 17:51:34.167208+01:00 |                              

# Accessing artifacts stored in S3
We have set the default artifact store in polyaxon as a S3 bucket in  (`s3://canotic-ai/polyaxon-storage/`)[https://s3.console.aws.amazon.com/s3/buckets/canotic-ai?prefix=polyaxon-storage/&region=us-east-1]

To access the artifacts, we have added a very simple example which prints the content of a text file stored in a sample folder.

In [3]:
!cat resources / artifacts.yaml

version: 1.1
kind: component
tags: [test]

run:
  init:
    - artifacts:
        dirs: [ "test-storage" ]
  kind: job
  container:
    image: ubuntu:latest
    workingDir: "{{ globals.artifacts_path }}/test-storage"
    command: ["cat", "random_text.txt"]
    imagePullPolicy: IfNotPresent


Run the job

In [4]:
!polyaxon run -f resources / artifacts.yaml -u -l

Creating a new run...
A new run `56e14c5646de46b8a15db42dddae3a0b` was created
You can view this run on Polyaxon UI: http://localhost:8000/ui/default/dev/runs/56e14c5646de46b8a15db42dddae3a0b/
[================================] 959021/959021 - 00:00:01
Artifacts uploaded
Run is approved
Starting logs for run: <Name: None> - <uuid: 56e14c5646de46b8a15db42dddae3a0b>
warning
running
2022-02-22 13:50:04.610585+01:00 | This is some random text for test. Test Text.
succeeded


## Deploying a training instance using SDK functionalities

In [13]:
if os.path.exists(".AISave"):
    shutil.rmtree(".AISave")

### Adding an app ID to model
The following AI instance can be associated with an app. The `app_id` here is associated with ankit@super.ai, so make sure you use an app under your own user owner ID.

In [14]:
template = AITemplate(
    input_schema=Schema(),
    output_schema=Schema(),
    configuration=Config(),
    name="MnistTrainingTemplate",
    description="Template of Sample MNIST training",
    model_class="MnistModel",
    requirements=["tensorflow-gpu==2.3.0", "polyaxon"],
    artifacts={"run": "resources/runDir/run_this.sh"},
    code_path=["resources/runDir"],
)
ai = AI(
    ai_template=template,
    input_params=template.input_schema.parameters(),
    output_params=template.output_schema.parameters(),
    name="mnist_training",
    version=1,
    description="AI instance of sample MNIST training",
    app_id="c1bed1ac-6418-40e3-9ce0-15a17d1bc38c",
)

[04/21/22 19:54:10] INFO     Copying all code_path content - MainThread             ]8;id=1650563650.227714-93989;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:274

                    INFO     Saved model in .AISave/mnist_training/1 - MainThread   ]8;id=1650563650.2434611-658417;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:880

In [5]:
ai.push()

[03/21/22 11:36:39] WARNING  Model already exists in the DB and overwrite is not    ]8;id=1647858999.175818-860695;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:916
                             set. - MainThread

'ca7ef522-e115-4c32-ab91-308d218a111b'

In [6]:
ai.training_deploy(
    orchestrator=TrainingOrchestrator.AWS_EKS,
    training_parameters=TrainingParameters(
        model_save_path="/tmp", training_data="/tmp",
        hyperparameters=HyperParameterSpec(
            trainable=True,
            optimizer="adam",
            log_learning_rate=-3,
            epochs=10
        ),
        model_parameter=ModelParameters(
            conv1_size=32,
            conv2_size=64,
            hidden1_size=500,
            dropout=0.8
        ),
    )
)

[03/21/22 11:36:45] INFO     Base image                                            ]8;id=1647859005.6773949-385326;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:1253
                             'superai-model-s2i-python3711-cpu-seldon:1' found               
                             locally. - MainThread

                    INFO     No change in pip layer. Reusing old layers from image ]8;id=1647859005.68697-594684;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:1268
                             sha256:9c3e1b928e0bdc123ac0c35ad5c8678979de564ad97463           
                             f1045f28eb0d6119fe... - MainThread

                    INFO     Running 's2i build -E /Users/ankitmishra/Work/mysuper ]8;id=1647859005.692607-249832;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:1348
                             ai/superai-sdk-private/docs/examples/ai/.AISave/mnist           
                             _training/1/environment -v                                      
                             /Users/ankitmishra/.aws:/root/.aws -v                           
                             /Users/ankitmishra/.superai:/root/.superai -v                   
                             /Users/ankitmishra/.canotic:/root/.canotic                      
                             --incremental=True . mnist_training-pip-layer:1                 
                             mnist_training:1' - MainThread

[03/21/22 11:36:46] INFO     Built main container `mnist_training:1` - MainThread  ]8;id=1647859006.868445-715039;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:1290

                    INFO     Time taken to build: 1.21s - MainThread               ]8;id=1647859006.870785-712948;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:1291

...

Output()

[03/21/22 11:36:58] INFO      Image pushed successfully to 185169359328.dkr.e ]8;id=1647859018.30742-760866;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/dockerizer/__init__.py\__init__.py]8;;\:297
                             cr.us-east-1.amazonaws.com/models/dev/ca7ef522-e                
                             115-4c32-ab91-308d218a111b/mnist_training:1  -                  
                             MainThread

[03/21/22 11:36:59] INFO     Found existing template [meta_ai_training_template(id= ]8;id=1647859019.7574039-921471;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:329
                             '9789481b-9fc3-461c-b5dc-2c486e70b4a4',                         
                             properties='{}',                                                
                             created_at='2022-03-14T15:15:28.930547+00:00')] -               
                             MainThread

[03/21/22 11:37:01] INFO     Created training instance {'data':                  ]8;id=1647859021.3445802-266202;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/apis/meta_ai/model.py\model.py]8;;\:855
                             {'insert_meta_ai_training_instance_one': {'id':                 
                             '8c112110-9127-4235-b4b6-74d88c6ddcb1'}}} -                     
                             MainThread

                    INFO     Create training instance :                            ]8;id=1647859021.347361-968085;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:1531
                             8c112110-9127-4235-b4b6-74d88c6ddcb1 - MainThread

This should start a training job. We will be adding features to monitor the training progress soon.

In [4]:
if os.path.exists(".AISave/intel_image_classification"):
    shutil.rmtree(".AISave/intel_image_classification")

In [5]:
template = AITemplate(
    input_schema=Schema(),
    output_schema=Schema(),
    configuration=Config(),
    name="IntelImageClassificationTemplate",
    description="Template of Sample MNIST training",
    model_class="IntelImageClassification",
    requirements=[
        "tensorflow-gpu==2.3.0", 
        "polyaxon[s3]", 
        "scikit-learn", 
        "tqdm", 
        "opencv-python-headless", 
        "aiobotocore==2.2.0", 
        "aioitertools==0.10.0", 
        "botocore==1.24.21"
    ],
    artifacts={"run": "resources/runDir/run_this.sh"},
    code_path=["resources/runDir"],
)
ai = AI(
    ai_template=template,
    input_params=template.input_schema.parameters(),
    output_params=template.output_schema.parameters(),
    name="intel_image_classification",
    version=1,
    description="AI instance of sample MNIST training",
    app_id="c1bed1ac-6418-40e3-9ce0-15a17d1bc38c",
)

[04/27/22 11:51:51] INFO     Copying all code_path content - MainThread             ]8;id=1651053111.623428-170704;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:274

                    INFO     Saved model in .AISave/intel_image_classification/1 -  ]8;id=1651053111.639615-524074;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:880
                             MainThread

In [6]:
ai.push()

[04/27/22 11:51:53] WARNING  Model already exists in the DB and overwrite is not    ]8;id=1651053113.6739771-812519;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:923
                             set. - MainThread

'5d03ae31-5b69-4cc1-818b-ec20ae55b8b7'

In [9]:
ai.training_deploy(
    orchestrator=TrainingOrchestrator.AWS_EKS,
    training_parameters=TrainingParameters(
        model_save_path=f"{TrainingParameters.output_folder_template}/output-model", 
        training_data="test/intel-image.zip",
        hyperparameters=HyperParameterSpec(
            trainable=True,
            batch_size=128,
            validation_split=0.2,
            epochs=10
        ),
        model_parameter=ModelParameters(
            conv1_size=32,
            conv2_size=32,
            dense_size=128
        ),
    ),
    enable_cuda=True,
)

[04/27/22 12:57:44] INFO     Base image                                            ]8;id=1651057064.450716-837191;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:1269
                             'superai-model-s2i-python3711-cpu-seldon:1' found               
                             locally. - MainThread

                    INFO     No change in pip layer. Reusing old layers from image ]8;id=1651057064.464519-393705;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:1284
                             sha256:913784aec97b8a0dc3fa179bad9a964d7277d9041f9c90           
                             b98b59ffb562e786ba... - MainThread

                    INFO     Running 's2i build -E /Users/ankitmishra/Work/mysuper ]8;id=1651057064.470716-632449;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:1359
                             ai/superai-sdk-private/docs/examples/ai/.AISave/intel           
                             _image_classification/1/environment -v                          
                             /Users/ankitmishra/.aws:/root/.aws -v                           
                             /Users/ankitmishra/.superai:/root/.superai -v                   
                             /Users/ankitmishra/.canotic:/root/.canotic                      
                             --incremental=True .                                            
                             intel_image_classification-pip-layer:1                          
                             intel_image_classification:1' - MainThread

[04/27/22 12:57:45] INFO     Built main container `intel_image_classification:1` - ]8;id=1651057065.922623-596646;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:1306
                             MainThread

                    INFO     Time taken to build: 1.51s - MainThread               ]8;id=1651057065.927197-517964;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:1307

...

Output()

[04/27/22 13:01:03] INFO      Image pushed successfully to 161853812624.dkr.e ]8;id=1651057263.181536-286195;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/dockerizer/__init__.py\__init__.py]8;;\:297
                             cr.us-east-1.amazonaws.com/models/dev/5d03ae31-5                
                             b69-4cc1-818b-ec20ae55b8b7/intel_image_classific                
                             ation:1  - MainThread

[04/27/22 13:01:10] INFO     Creating new training run based on exising          ]8;id=1651057270.757547-940285;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/apis/meta_ai/model.py\model.py]8;;\:915
                             template: meta_ai_training_template(id=9873734f-53f             
                             9-4a4c-a5cd-de45bfa472b1, properties={},                        
                             created_at=2022-04-21T14:53:13.513495+00:00,                    
                             description=None) - MainThread

[04/27/22 13:01:11] INFO     Created training instance {'data':                  ]8;id=1651057271.389445-130006;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/apis/meta_ai/model.py\model.py]8;;\:931
                             {'insert_meta_ai_training_instance_one': {'id':                 
                             '3670d6db-7b3a-4d6e-a6e5-9307d9bf30fa'}}} -                     
                             MainThread

                    INFO     Create training instance :                            ]8;id=1651057271.3927429-199153;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:1544
                             3670d6db-7b3a-4d6e-a6e5-9307d9bf30fa - MainThread